# Canary

The following notebook is intended to explain the process of how to create the card game named 'Canary'.

#### Game Setup
- The game can accommodate 2-8 players.
- For 2-4 players, a single deck with 54 cards (including 2 Jokers) is used. For 5-8 players, two decks (108 cards total) are used.
- At the start, each player is dealt 3 face-down cards, which are to remain facedown, and an additional 6 cards. From the additional 6 cards, they choose 3 to place face-up for everyone to see. The remaining 3 cards remain in their hand.
#### Objective
- The goal of the game is to be the first player to play all of your cards (First from hand, then from face-up, and lastly the face-down).
#### Game Play
- The game begins by determining the starting player, which is based on who has the lowest card, starting from 3s and moving upward as needed. Ties are determined by who has the most of the worst card.
- Players take turns in a clockwise order.
- On their turn, a player must play a card that beats the card played before it.
- If a player cannot play a card, or does not wish to play a card, they must pick up the cards in the played pile. 
- Players can play multiple cards of the same value at the same time.
- If a player has less than 3 cards in their hand and the draw pile isn't empty, they must draw until they have 3 cards.
- Once a player has no cards in their hand, and the draw pile is empty, they can start playing their face-up cards. Once no cards remain in their hand or faceup pile, they must choose a face-down card to turn over. If the card beats the previous card, they may play it. If it does not, they must pick up the pile and the card they just turned over.
#### Special Cards and Rules
- 2: Can be played on any card. Allows the player to take another turn.
- 7: The next card played must be 7 or lower. 7s must be played in numerical order.
- 10: Can be played on any card. Clears the pile of cards played before it.
- Ace: The highest value card.
- Joker: Can be played on any card. Reverses the order of play.
- If the same value card is played 4 times in a row, the pile is cleared.
#### Winning the Game
- The first player to play all of their cards (hand, face-up, and face-down) is declared the winner and is called the "Canary".

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


#### Code

The first task is to create the initial setup for the game. We need to create a deck of cards and distribute the cards to each of the players in the correct manner.

The code will use the following library

In [55]:
import itertools
import random
import json
import pandas as pd

#### Card Class

The card class is used in the creation of the deck of cards. Each deck will contain an array of these card objects which all contain a value and suit. Additionally if the cards is a 2, 10, or Joker its is_special value will be a 1. If the card is a 7, the is_seven value is given a 1. These attributes are used throughout the code to help with various logic of the game. the __repr__ function is helpful for printing the values of the card.

In [56]:
class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
        # Determine if the card is special
        self.is_special = value in [2, 10, 'Joker']
        self.is_seven = value in [7]

    def __repr__(self):
        return f"{self.value} of {self.suit}" if self.suit else f"{self.value}"


#### CardGame Class

The CardGame class object is how we keep track of everything that happens in the game. To create a CardGame object simply pass in the number of players. This CardGame object will then be manipulated by many other functions to simulate the game.

In [57]:
class CardGame:
    def __init__(self, num_players):
        self.num_players = num_players
        self.deck = []
        self.players_hands = [[] for _ in range(num_players)]
        self.face_down_cards = [[] for _ in range(num_players)]
        self.face_up_cards = [[] for _ in range(num_players)]
        self.draw_pile = []
        self.play_pile = []
        self.game_over = False
        self.play_direction = 1 # 1 for clockwise, -1 for counterclockwise
        self.current_player_index = -1
        self.turn = 0
        self.known_opponent_cards = [[] for _ in range(num_players)]
        self.game_number = 0

In [58]:
card_game = CardGame(4)

### Creating the Deck

In order to play the game you need a deck of cards. This deck of cards is a standard deck with the inclusion of two joker cards. If there are more than 4 players, then two decks are used. The Joker card does not contain have a suit.

In [59]:
def create_deck(self):
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        values = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]
        deck = [Card(value, suit) for suit in suits for value in values]

        # Include Jokers without a suit
        deck.extend([Card('Joker', None) for _ in range(2)])  # Adding two Jokers

        # Use 2 decks for 5-8 players
        if self.num_players >= 5:
            deck = deck * 2

        return deck


card_game.deck = create_deck(card_game)
pd.DataFrame(card_game.deck).head(10)

,0
0,2 of Hearts
1,3 of Hearts
2,4 of Hearts
3,5 of Hearts
4,6 of Hearts
5,7 of Hearts
6,8 of Hearts
7,9 of Hearts
8,10 of Hearts
9,11 of Hearts


#### Game Setup

The setup_game function sets the game up so that the first turn may be executed. The cards are first randomly shuffled. Then, cards are dealt one at a time to each of the players until they all have 3 facedown, and 6 face up candidate cards. Then, from the face up candidate cards each player randomly chooses 3 to be face up, the other 3 become their player hand. For now, the face up cards are chosen randomly from the face up candidate cards. This would normally be chosen by each player via their strategy. Once the cards have been dealt, the remaining cards become the draw pile. 

In [60]:
def setup_game(self):
        # Shuffle the deck
        random.shuffle(self.deck) 

        # Deal 3 face-down and 6 to choose face-up cards to each player
        for i in range(self.num_players):
            # Deal 3 face-down cards
            self.face_down_cards[i] = [self.deck.pop() for _ in range(3)]

            # Deal 6 cards for choosing 3 to be face-up
            face_up_candidates = [self.deck.pop() for _ in range(6)]

            # Players choose 3 of these to be face-up, for simplicity, randomly select here
            self.face_up_cards[i] = random.sample(face_up_candidates, 3)

            # The remaining 3 cards from the 6 initially dealt for choosing go into the player's hand
            self.players_hands[i] = [card for card in face_up_candidates if card not in self.face_up_cards[i]]

        # The rest of the cards form the draw pile
        self.draw_pile = self.deck

setup_game(card_game)
print("Facedown Cards: ",card_game.face_down_cards)
print("Faceup Cards: ",card_game.face_up_cards)
print("Player Hands: ",card_game.players_hands)
print("Draw Pile: ",card_game.draw_pile)

Facedown Cards:  [[10 of Hearts, 9 of Diamonds, 5 of Clubs], [9 of Spades, 3 of Clubs, 7 of Spades], [4 of Hearts, 11 of Hearts, 13 of Hearts], [13 of Diamonds, 9 of Clubs, 12 of Spades]]
Faceup Cards:  [[9 of Hearts, 8 of Clubs, 15 of Hearts], [14 of Spades, 3 of Diamonds, 7 of Hearts], [8 of Spades, 12 of Clubs, 8 of Diamonds], [2 of Hearts, 11 of Spades, 4 of Spades]]
Player Hands:  [[7 of Clubs, 13 of Spades, 8 of Hearts], [13 of Clubs, 6 of Hearts, 14 of Hearts], [2 of Clubs, 4 of Clubs, 2 of Spades], [Joker, 6 of Spades, 10 of Spades]]
Draw Pile:  [2 of Diamonds, 5 of Spades, 10 of Diamonds, 4 of Diamonds, 6 of Diamonds, 3 of Spades, 11 of Diamonds, 11 of Clubs, 12 of Hearts, Joker, 15 of Clubs, 6 of Clubs, 5 of Diamonds, 15 of Spades, 14 of Diamonds, 14 of Clubs, 3 of Hearts, 10 of Clubs, 7 of Diamonds, 12 of Diamonds, 5 of Hearts, 15 of Diamonds]


#### Determining Who Goes First

To determine who goes first each hand must be examined. The player with the lowest card (3) shall go first. If multiple players have a 3, then the player with the most 3s shall go first. If that is a tie, and they both don't have all 3s, then the player with the most 4s shall go first, if that is a tie, then 5s, then 6s, ect. If they have the exact same hand, then a player is chosen randomly.

In [61]:
def find_starting_player(self):
        # Define the order for determining the starting player
        card_value_order = [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 'Joker', 10, 2]

        # Initialize counts for each card value for each player
        player_card_counts = [{value: 0 for value in card_value_order} for _ in range(self.num_players)]

        # Count the number of each card value in each player's hand
        for player_index, hand in enumerate(self.players_hands):
            for card in hand:
                if card.value in card_value_order:
                    player_card_counts[player_index][card.value] += 1

        # Determine who starts based on the cards
        for value in card_value_order:
            # Find players with the current card value
            players_with_card = [(index, count[value]) for index, count in enumerate(player_card_counts) if count[value] > 0]

            if not players_with_card:  # If no player has the card, continue to the next value
                continue

            # If only one player has the card, they start
            if len(players_with_card) == 1:
                return players_with_card[0][0]

            # If multiple players have the same card, compare counts
            max_count = max(players_with_card, key=lambda x: x[1])[1]
            players_with_max = [player for player, count in players_with_card if count == max_count]

            if len(players_with_max) == 1:
                return players_with_max[0]
            # If still tied, continue to the next card value

        # If somehow no one can start, return a random player as fallback
        return random.randint(0, self.num_players - 1)


print("Initial value of current_player_index: ", card_game.current_player_index)
print("Player Hands: ", card_game.players_hands)
print("The player with the lowest card(s) goes first. Player numbers are 0,1,2,3")
card_game.current_player_index = find_starting_player(card_game)
print("Value of current_player_index after running find_starting_player(): ", card_game.current_player_index)



Initial value of current_player_index:  -1
Player Hands:  [[7 of Clubs, 13 of Spades, 8 of Hearts], [13 of Clubs, 6 of Hearts, 14 of Hearts], [2 of Clubs, 4 of Clubs, 2 of Spades], [Joker, 6 of Spades, 10 of Spades]]
The player with the lowest card(s) goes first. Player numbers are 0,1,2,3
Value of current_player_index after running find_starting_player():  2


#### Simulating the Game

The next few functions all stem from the 'start_game()' function. This function begins with a while statement that runs while the game is not over. If the game is not over, then the data of the CardGame object is saved via the collect_game_data() function. This function will be explained in a following potion of the notebook. After the data collection, there is a check to make sure that the game is not over. If the game is over the while loop will end. Otherwise, the 'play_turn()' function is called in which a player will be able to play a card (or not).

In [62]:
def start_game(self):
        
    while not self.game_over:
        # Player takes their turn.
        game_data = collect_game_data(self)
        #print(game_data)

        self.game_over = check_game_over(self)
        
        play_turn(self)        
              
    print("Game Over!")

#### Playing a Turn

For every turn played, there is a counter that keeps track of the number of turns. 

For each turn, an attempt is made to first play from the hand, then the face up cards, then the face down cards. The term 'attempt' is referring to the number of cards in each pile. If a player cannot 'attempt' to play a card from their hand because it is empty, then they must attempt to play from the face up pile. If that pile is also empty, then they should attempt to play from the facedown pile, and if thats empty then the game should end. 

If there are cards to be played then each of the attempt functions will return True, which will then cause the program to exit the if statements. If the attempt function returns true, it does NOT mean the card can be played. The test to see if a card can be played will happen at a later point. 

In [63]:
def play_turn(self):
    self.turn += 1
    #print(f"Player {self.current_player_index + 1}'s turn:")
        

    # Try playing from hand, face-up, then face-down in order.
    if not attempt_play_from_hand(self):
        if not attempt_play_from_face_up(self):
            if not attempt_play_from_face_down(self):
                return 

#### Attempting Play From Each Hand

The first attempt function is the 'attempt_play_from_hand' function. This function simply calls  'attempt_play'. The 'attempt_play' function is passed the cards in the current players hand. 

Likewise, 'attempt_play_from_face_up' also calls 'attempt_play' and passes the current player's face up cards. Additionally, a boolean value 'is_face_up' is passed and set to True. This value helps us later understand which function called 'attempt_play', which is important for determining which pile we are playing the card from. Perhaps this could be simplified?

'attempt_play_from_face_down' has similar logic. Instead of the entire face down pile being passed, and random card from the pile is passed. Similarly, a True 'is_face_down' boolean is passed.

In [64]:
def attempt_play_from_hand(self):
        #print("Player Hand: ", self.players_hands[self.current_player_index])
        return attempt_play(self, card_source = self.players_hands[self.current_player_index])

In [65]:
def attempt_play_from_face_up(self):
        #print("Players Face-Up Hand: ", self.face_up_cards[self.current_player_index])
        return attempt_play(self, card_source = self.face_up_cards[self.current_player_index], is_face_up = True)

In [66]:
def attempt_play_from_face_down(self, is_face_down = True):
        # Face-down play is a bit different as it involves random choice and immediate play without checking
        if self.face_down_cards[self.current_player_index]:
            chosen_card = random.choice(self.face_down_cards[self.current_player_index])
            ##print("Players Face-Down Hand:: ", chosen_card)
            return attempt_play(self, card_source = chosen_card, is_face_down = True)
        else:
            return False

### Attempt Play

The goal of this function is to determine if card_source contains cards. If they do, then another function called 'can_play_card' is called to determine if the cards are playable, meaning they beat the current top card in the play pile. If there are no playable cards, then the pile is picked up by the current player, and True is returned. 

It is important to remember where True is being returned to. It goes all the way back to play_turn(), which means the turn is completed and the code can continue.
If the card_source was empty, then False is returned, which would cause the code to enter the next if statement, leading to the next hand being used to attempt a play. 

If there are playable cards, then a random card is chosen and passed into a function called play_card().

If is_face_up or is_face_down True an if statement is entered. These if statements again help with future logic and understanding where the play attempt originated from. 


Lastly, True is returned to end the turn.


In [67]:
def attempt_play(self, card_source, is_face_up = False, is_face_down = False):

        if is_face_up:
            if not card_source:
                #Face up cards is empty, move to facedown
                return False
            
            playable_cards = [card for card in card_source if can_play_card(self, card)]
            #print("Playable Cards: ", playable_cards)
            if not playable_cards:
                    #print("No playable cards in face up pile")
                    pick_up_pile(self)
                    return True
            chosen_card = random.choice(playable_cards)
            play_card(self, chosen_card, is_face_up = True)
            return True
        
        if is_face_down:
            if not card_source:
                return False
            
            #print("Card Selected: ", card_source)
            
            if can_play_card(self, card_source):
                self.play_pile.append(card_source)
                self.face_down_cards[self.current_player_index].remove(card_source)
                return True
            else:
                self.play_pile.append(card_source)
                self.face_down_cards[self.current_player_index].remove(card_source)
                pick_up_pile(self)
                return True

        
        if not card_source:
            #print("No Cards in Hand!")
            return False
        
        playable_cards = [card for card in card_source if can_play_card(self,card)]
        #print("Playable Cards: ", playable_cards)
        if not playable_cards:
            pick_up_pile(self)   
            return True

        chosen_card = random.choice(playable_cards)
        play_card(self, chosen_card)
        return True

#### Can a Card Be Played?

To determine if a card can be played, the card must beat the current top card in the play pile.

In [68]:
def can_play_card(self, card):
        """Check if the card can be played on top of the play pile."""
        if not self.play_pile:
            return True  # Any card can be played if the play pile is empty
        
        top_card = self.play_pile[-1]
        
        # Allow any card to be played if the top card is a special card or the played card is special
        if top_card.is_special:
            return True
        
        if card.is_special:
            return True
        
        if top_card.is_seven:
            #print(card.value, " <= ", top_card.value,"?", card.value <= top_card.value)
            return card.value <= 7
        
        #print(card.value, " >= ", top_card.value,"?", card.value >= top_card.value)
        return card.value >= top_card.value

#### Playing a Card

For future projects I have included an additional attribute to the CardGame object called known_opponent_cards. This attribute is an array of the same length as player_hands except instead of the card suits and values, it reads 'unknown' unless the card has been played and picked up by a player. Consider a real life game of canary, since everyone in real life can see the cards being played, they can theoretically, with perfect memory, remember all of the cards played. Therefore, if a player picks up these cards they are 'known' cards. 'known_opponent_cards is simply' a reflection of these cards and this knowledge.

The relevancy of this attribute to the play_card function is if the card is played and it is one of the known opponent cards, then it should be removed from this list since the opponent no longer holds a known card.

Following this if a series of if statements checking if the card is special or is a seven, and if it is there is a series of logic to plays out the rules from the game. 

A card is played by adding it to the play pile and removing it from whatever player's pile it came from.

Following the playing of a card, a function called post_play_card_actions is called.

In [69]:
def play_card(self, chosen_card, is_face_up=False, is_face_down=False):
        # Remove the card from known_opponent_cards if it was known
        if chosen_card in self.known_opponent_cards[self.current_player_index]:
            self.known_opponent_cards[self.current_player_index].remove(chosen_card)

        # Check for special cards (Joker, 2, 10, and handling of 7 if needed)
        if chosen_card.is_special:
            if chosen_card.value == 'Joker':
                self.play_pile.append(chosen_card)
                #print(f"Played {chosen_card}.")
                self.play_direction *= -1
                if not is_face_down:
                    if is_face_up:
                        self.face_up_cards[self.current_player_index].remove(chosen_card)
                    else:
                        self.players_hands[self.current_player_index].remove(chosen_card)
                        
                post_play_card_actions(self)
                return
            elif chosen_card.value == 2:
                # Logic for 2 allowing the player to play again
                self.play_pile.append(chosen_card)
                #print(f"Played {chosen_card}.")
                if not is_face_down:
                    if is_face_up:
                        self.face_up_cards[self.current_player_index].remove(chosen_card)
                    else:
                        self.players_hands[self.current_player_index].remove(chosen_card)
                allow_play_again(self)
                post_play_card_actions(self)
                return
            elif chosen_card.value == 10:
                # Maybe a special logic for 10, like clearing the play pile
                self.play_pile.append(chosen_card)
                #print(f"Played {chosen_card}.")
                clear_play_pile(self)
                # Remove the card from its current location unless it's face-down
                if not is_face_down:
                    if is_face_up:
                        self.face_up_cards[self.current_player_index].remove(chosen_card)
                    else:
                        self.players_hands[self.current_player_index].remove(chosen_card)
                        
                post_play_card_actions(self)
                return
            # Additional logic for any special actions based on card
        elif chosen_card.is_seven:
            # Handle seven's special rule if applicable
            self.play_pile.append(chosen_card)
            #print(f"Played {chosen_card}.")
            # Remove the card from its current location unless it's face-down
            if not is_face_down:
                if is_face_up:
                    self.face_up_cards[self.current_player_index].remove(chosen_card)
                else:
                    self.players_hands[self.current_player_index].remove(chosen_card)
            post_play_card_actions(self)
            return

        if not is_face_down:
            if is_face_up:
                self.face_up_cards[self.current_player_index].remove(chosen_card)
            else:
                self.players_hands[self.current_player_index].remove(chosen_card)

        self.play_pile.append(chosen_card)
        #print(f"Played {chosen_card}.")
        post_play_card_actions(self)

#### Post Play Actions

Following a card being played, the player playing the card needs to draw a new card until they have at least 3 cards in their hand.

Next, the function checks if there exists 4 of the same card values in a row in the play_pile. If there are, then the play_pile is cleared. 

Lastly, the current player index is changed to the next player depending on the number of players and the current play direction.

In [70]:
def post_play_card_actions(self):
        # Ensures the player has at least 3 cards in their hand if the draw pile isn't empty
        while len(self.players_hands[self.current_player_index]) < 3 and self.draw_pile:
            draw_card(self)
            #print(f"Player {self.current_player_index + 1} draws a card. Hand now: {self.players_hands[self.current_player_index]}")
        
        if len(self.play_pile) >= 4:
                #print("thats a big pile")
                for i in range(len(self.play_pile) -3):
                    if self.play_pile[i] == self.play_pile[i+1] == self.play_pile[i+2] == self.play_pile[i+3]:
                        #print("4 in a row!")    
                        self.play_pile.clear()
                
            
        # Move to the next player
        self.current_player_index = (self.current_player_index + self.play_direction) % self.num_players   

#### Reversing Play

If a joker is played, the order of play is changed. 
If is was going clockwise, it is now counter-clockwise. 
If it was counter-clockwise, it is now clockwise.

In [71]:
def reverse_play_order(self):
        """Reverses the current order of play."""
        # This is a simplified representation. You might store the current direction of play
        # (e.g., as a class attribute) and reverse it here. For example:
        self.play_direction *= -1
        #print("Reversing Direction!")
        return

#### Drawing Cards

If a played has less than 3 cards they need to draw cards until they do.

In [72]:
def draw_card(self):
        """Player draws a card from the draw pile."""
        if self.draw_pile:
            card_drawn = self.draw_pile.pop()
            self.players_hands[self.current_player_index].append(card_drawn)
        else:
            print("The draw pile is empty. No card drawn.")

#### Picking Up The Pile

When a played cannot play a card then must pick up the pile. The play pile is added to the current players hand. Additionally the known_opponent_cards is updated to refelct the cards that were picked up. And lastly the play pile is cleared since the cards were all picked up.

In [73]:
def pick_up_pile(self):
        """Player picks up the play pile."""
        self.players_hands[self.current_player_index].extend(self.play_pile)
        self.known_opponent_cards[self.current_player_index].extend(self.play_pile)
        self.play_pile.clear()
        post_play_card_actions(self)
        #print("Pile picked up!")

#### Checking If The Game Is Over

To check if the game is over, each hand of each player is checked to see if it is empty. If all hands of a player are empty then they are the winner! If no player has all empty hands then the game is not over.

In [74]:
def check_game_over(self):
        # Implement logic to determine if the game has ended.
        # This could involve checking if any player has successfully played all their cards (hand, face-up, and face-down).
        for player_index in range(self.num_players):
            if (not self.players_hands[player_index] and 
                not self.face_up_cards[player_index] and 
                not self.face_down_cards[player_index]):
                print(f"Player {player_index + 1} wins and is declared the Canary!")
                return True  # A winner is found
        return False  # The game continues

#### A Two is Played

If a two is played then the player who played the two gets to go again. This played is not allowed to draw a card. They must play again from the cards they have, even if they have less than 3 cards. This is the only exception to the rule. 

In [75]:
def allow_play_again(self):
        #print("Play again:")
        self.current_player_index = (self.current_player_index - self.play_direction) % self.num_players
        #print(self.current_player_index)
        # Implement logic to allow the current player to play again. This might mean not advancing
        # the current_player_index or setting a flag that allows another turn for the current player.

#### Clearing The Pile

Pretty straight forward. Either a 10 was played, 4 of the same value card were played in a row, or someone picked up the pile. Either way the play pile is cleared. 

In [76]:
def clear_play_pile(self):
        #print("Clearing play pile")
        self.play_pile.clear()
        # Implement logic to clear the play pile if a 10 is played, for example.

#### Collecting Data About The Game

After each turn the CardGame object attributes are saved to a text file in a JSON format. This data can be used for later projects such as AI players.

In [77]:
def collect_game_data(self):
        game_data = {
            'game_number': self.game_number,
            'turn_number': self.turn,
            'current_player': self.current_player_index + 1,
            'num_players': self.num_players,
            'cards_in_hands': [[str(card) for card in hand] if i == self.current_player_index else ['Unknown'] * len(hand) for i, hand in enumerate(self.players_hands)],
            'known_opponent_cards': [[str(card) for card in known_cards] for known_cards in self.known_opponent_cards], 
            'num_cards_in_hands': [len(hand) for hand in self.players_hands],
            'top_card': str(self.play_pile[-1]) if self.play_pile else None,
            'cards_in_pile': [str(card) for card in self.play_pile],
            'face_up_cards': [[str(card) for card in face_up] for face_up in self.face_up_cards],
            'num_face_up_cards': [len(face_up) for face_up in self.face_up_cards],
            'num_face_down_cards': [len(face_down) for face_down in self.face_down_cards],
            'play_direction': self.play_direction,
            'num_cards_in_draw_pile': len(self.draw_pile),
            'game_over': self.game_over
        }

        with open('game_data.txt', 'a') as file:
            json.dump(game_data, file)
            file.write('\n\n')

        return game_data

#### Final Statement

And thats it! To start the game simply call start_game and pass in a CardGame object. The function will output the game data from each turn.

In [80]:
game_number = 1
while game_number <= 3:    
    print("Game: ", game_number)
    game = CardGame(4)
    game.deck = create_deck(game)
    setup_game(game)
    game.current_player_index = find_starting_player(game) 
    game.game_number = game_number 
    start_game(game)
    game_number += 1

Game:  1
Player 3 wins and is declared the Canary!
Game Over!
Game:  2
Player 4 wins and is declared the Canary!
Game Over!
Game:  3
Player 2 wins and is declared the Canary!
Game Over!
